# Post-Proccessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from scipy import signal
from scipy import misc # pip install Pillow
# import imageio
import matplotlib.pylab as pylab
from PIL import Image
import tarfile
import bz2
import zipfile

%matplotlib inline
pylab.rcParams['figure.figsize'] = (20.0, 7.0)

#### Helper Functions

In [2]:
#### Global Variables



rgb_to_YCbCr = np.array([[0.29900, 0.58700, 0.11400],
                       [-0.16874, -0.33126, 0.50000],
                       [0.50000, -0.41869, -0.08131]])

YCbCr_to_rgb = np.linalg.inv(rgb_to_YCbCr)


###common qunatizing matrices for dct 

Ql = np.array( [[16,12,14,14,18, 24, 49, 72],
                [11, 12, 13, 17, 22, 35, 64, 92],
                [10, 14, 16, 22, 37, 55, 78, 95],
                [16, 19, 24, 29, 56, 64, 87, 98],
                [24, 26, 40, 51, 68, 81, 103, 112],
                [40, 58, 57, 87, 109, 104, 121, 100],
                [51, 60, 69, 80, 103, 113, 120, 103],
                [61, 55, 56, 62, 77, 92, 101, 99]]
                ).T

Qc = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
              [18, 21, 26, 66, 99, 99, 99, 99],
              [24, 26, 56, 99, 99, 99, 99, 99],
              [47, 66, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99]]).T


In [3]:
def PSNR(ref, meas, maxVal=255):
    assert np.shape(ref) == np.shape(meas), "Test image must match measured image dimensions"
        
    
    dif = (ref.astype(float)-meas.astype(float)).ravel()
    mse = np.linalg.norm(dif)**2/np.prod(np.shape(ref))
    psnr = 10*np.log10(maxVal**2.0/mse)
    return psnr


#####################################################################################


#DCT Functions 
def dct2(a):
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

#####################################################################################

##Padds an image so that its dimensions are divisible by 8.
def zero_pad(img):

    new_img = img
    
    if img.shape[0] % 8 != 0:
        zeros = np.zeros((8 - img.shape[0] % 8,img.shape[1],img.shape[2]))
        new_img = np.vstack((img,zeros))

    temp = new_img

    if img.shape[1] % 8 != 0:
        zeros = np.zeros((new_img.shape[0],8 - img.shape[1] % 8,new_img.shape[2]))
        temp = np.hstack((new_img,zeros))
    
    print("Zero Padded Shape: ", temp.shape)

    return temp

#####################################################################################

def helper_divide(A,Q):
    
    Y = np.zeros((8,8))
    for u, v in [(u, v) for u in range(8) for v in range(8)]:
        Y[u, v] = round(A[u, v] / Q[u, v])

    return Y 
def helper_mul(A,Q):
    
    Y = np.zeros((8,8))
    for u, v in [(u, v) for u in range(8) for v in range(8)]:
        Y[u, v] = A[u, v] * Q[u, v]

    return Y 

##Makes Sure that the values of the img are integers.
def quantize(U, q = 50.):

    s = U.shape
    Y = np.zeros(s)

    if q <= 50 and q > 0:

        alpha = 50.0 / q
    else:
        alpha = 2 - q / 50.0

    for i in r_[:U.shape[0]:8]:
        for j in r_[:U.shape[1]:8]:
            Y[i:(i+8),j:(j+8),0] = helper_divide(U[i:(i+8),j:(j+8),0], Ql)
#             Y[i:(i+8),j:(j+8),0] = (U[i:(i+8),j:(j+8),0] // (Ql))
    for k in range(1,3):
        for i in r_[:U.shape[0]:8]:
            for j in r_[:U.shape[1]:8]:
                Y[i:(i+8),j:(j+8),k] = helper_divide(U[i:(i+8),j:(j+8),k], alpha*Qc)
#                 Y[i:(i+8),j:(j+8),k] = (U[i:(i+8),j:(j+8),k] // (alpha*Qc))

    

    return Y

def iquantize(U, q=50.):
    
    s = U.shape
    Y = np.zeros(s)
    
    if q <= 50 and q>= 1:
        alpha = 50./q
    else:
        alpha = 2-q/50.0
    
    for i in r_[:U.shape[0]:8]:
        for j in r_[:U.shape[1]:8]:
#             Y[i:(i+8),j:(j+8),0] = (U[i:(i+8),j:(j+8),0] * (Ql))
            Y[i:(i+8),j:(j+8),0] = helper_mul(U[i:(i+8),j:(j+8),0], alpha*Ql)
    for k in range(1,3):
        for i in r_[:U.shape[0]:8]:
            for j in r_[:U.shape[1]:8]:
                Y[i:(i+8),j:(j+8),k] = helper_mul(U[i:(i+8),j:(j+8),k], alpha*Qc)
#                 Y[i:(i+8),j:(j+8),k] = (U[i:(i+8),j:(j+8),k] * (alpha* Qc))

    
    return Y


#####################################################################################


##converts from RGB To YCbCr 
def rgb2YCbCr(U):
    s = U.shape

    Y = np.zeros(s)
    for i in range(s[0]):
        for j in range(s[1]):

            Y[i,j] = np.dot(rgb_to_YCbCr,U[i,j]) + np.array([[0],[128],[128]])[:,0]
    return Y


##converts from YCbCr To RGB
def YCbCr2rgb(U):
    s = U.shape
    Y = np.zeros(s)
    for i in range(s[0]):
        for j in range(s[1]):
            Y[i,j] = np.dot(YCbCr_to_rgb,U[i,j] -  np.array([[0],[128],[128]])[:,0]) 
    return Y


#####################################################################################



## Zipping\unzipping the image with some extra info 

## Zipping\unzipping the image with some extra info 

def zipping(img,min_val,max_val,beta,og_shape,img_name):

    ##creates a txt file that contains some extra info on the image such as dimensions. 
    output = open("rec_info.txt", "w")
    output.write(img_name + "|" + str(og_shape[0]) + "|" + str(og_shape[1]) + "|" + str(og_shape[2])+ "|" + str(min_val) + "|" + str(max_val) + "|" + str(beta))
    output.close()

    
    ##save Image 
    scipy.misc.imsave("rec.tiff",np.uint8(img))

    
    ###Encode Image 
#     tar = tarfile.open("rec.tar.bz", "w:bz2")
#     tar.add("rec.tiff")
#     tar.add("rec_info.txt")
    
#     tar.close()


    zipf = zipfile.ZipFile('reczip.zip', 'w', zipfile.ZIP_DEFLATED)
    zipf.write("rec.tiff")
    zipf.write("rec_info.txt")
    zipf.close()


    
#     compressed_img_bytes = os.stat("rec.tar.bz").st_size


    compressed_img_bytes = os.stat("reczip.zip").st_size
    if (compressed_img_bytes > 8000):
        print("Zipped File Too Big! " + str(compressed_img_bytes) + " Bytes")
    
    print("image zipped")

    return min_val, max_val


def unzipping():

    ## Unpack 
#     tar = tarfile.open("rec.tar.bz", "r")
#     tar.extractall()
#     tar.close()
    zip_ref = zipfile.ZipFile("reczip.zip", 'r')
    zip_ref.extractall()
    zip_ref.close()
    
    info = open("rec_info.txt", "r")
    info = info.read()
    info = info.split("|")
    print(info)
    
    img_name = info[0]
    og_shape = (int(info[1]),int(info[2]),int(info[3]))
    min_val = info[4]
    try:
        min_val = float(min_val)
    except:
        min_val = True
    
    max_val = float(info[5])
    beta = float(info[6])
    

    r = Image.open("rec.tiff")
    r = np.array(r)
    if min_val != True:
        r = ((r * max_val)/ 255) + min_val

    
    print("image unzipped")
    r = r.astype(int)
#     print(r)
    return r, img_name, og_shape, min_val, max_val, beta


### Post-Processing

In [4]:
def recon_image(dct_im,q,min_val,max_val, og_shape):

    if min_val == True:
        dct_im = scipy.misc.imresize(dct_im, og_shape, interp='bilinear', mode=None)

        return dct_im
    
    else:
        #Inverse Quantization
        print("Inverse Quantize")
        dct_im = iquantize(dct_im,q)
    

        recon_im = np.zeros(dct_im.shape)  
        for i in r_[:dct_im.shape[0]:8]:
            for j in r_[:dct_im.shape[1]:8]:
                recon_im[i:(i+8),j:(j+8)] = idct2( dct_im[i:(i+8),j:(j+8)] )



#       print("recon_img shape: ",recon_im.shape)
                
        recon_im = YCbCr2rgb(recon_im)       
        recon_im[np.where(recon_im > 255)] = 255
        recon_im[np.where(recon_im < 0)] = 0


        #convert back to RGB
        print("converted back to RGB")
        rgb_im = np.uint8(recon_im)
        
        #Upsample
        recon_im = scipy.misc.imresize(recon_im, og_shape, interp='nearest', mode=None)
        
    return recon_im

### Main 

In [ ]:
## Unzip the compressed file.
rec, img_name, og_shape, min_val, max_val, beta = unzipping()

## Reconstruct the image  
recon_im = recon_image(rec, beta,min_val,max_val,og_shape)





##Original Image
path = "createrLake.tiff"
original_img = misc.imread(path)


plt.imshow( np.hstack((original_img,recon_im)))
print("PSNR: " + str(PSNR(original_img,recon_im)) + "db")

['createrLake.tiff', '432', '1200', '3', '-47.0', '174.0', '50.0']
image unzipped
Inverse Quantize
converted back to RGB
PSNR: 25.9234471643db
